Nous allons dans notre projet avoir deux manières différentes de répondre à notre problèmatique qui est de détecter les grand feu qui sont les plus dangereux et dévastateur pour la planète afin d'aider les pompiers à connaître l'urgence de la situation en cas de départ de feu

Ici nous chargons notre jeu de donnée, nous avions initialement un jeu de donné qui était entre 1992 et 2020 mais nous avons réalisé qu'il y avait de nombreuses valeurs manquantes dans les premieres années du data set c'est pour cela que nous avons décidé de nous concentré sur l'année 2020. (d'autant plus que le data set complet mettais 20 minutes à se charger)

In [2]:
import pandas as pd
df=pd.read_csv("2020_FPA_FOD_cons.csv")

C:\Users\axjui\AppData\Local\Temp\ipykernel_20032\2025074942.py:2: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("2020_FPA_FOD_cons.csv")


Le script cleaning.py est un pipeline complet de préparation de données spécialement conçu pour transformer des données brutes (probablement des relevés d'incendies et météo) en un format mathématique propre, digeste et optimisé pour un algorithme comme XGBoost.

Voici l'analyse détaillée de la fonction finale full_data_cleaning_pipeline et comment elle orchestre les autres fonctions pour préparer le terrain au Machine Learning.

La Fonction Maîtresse : full_data_cleaning_pipeline
Cette fonction agit comme un chef d'orchestre. Elle prend le DataFrame brut en entrée et exécute 6 étapes séquentielles critiques. Si vous lancez cette fonction, voici ce qui arrive à vos données :

Étape 1 : Le "Grand Ménage" (clean_dataframe)
Action : Elle supprime les colonnes qui sont trop vides (plus de 50% de NaN) ou qui ne servent à rien (colonnes constantes où toutes les valeurs sont identiques)..

Étape 2 : Correction des Erreurs de Capteurs (handle_aberrant_values)
Action : Elle détecte des valeurs impossibles spécifiques aux capteurs météo/satellites (ex: -1e30, codes erreur -900, erreurs topo 32767). Elle remplace ces erreurs par NaN (valeur manquante).

Étape 3 : Nettoyage Catégoriel et Temporel (clean_categorical_features)
Action :

Dates : Transforme DISCOVERY_DATE en chiffres utilisables : Mois et Jour de l'année (saisonnalité).

Nettoyage : Supprime les identifiants uniques (FIRE_NAME, FPA_ID) qui causeraient du sur-apprentissage (overfitting) immédiat.

Conversion : Force les colonnes qui ressemblent à du texte mais sont des chiffres (ex: NDVI) à devenir numériques.

Gestion des NaN : Remplit les trous dans les catégories par le mot "Missing".

Étape 4 : Imputation et Anti-Fuite (impute_nan_and_drop_manual_cols)
Action :

Anti-Fuite (Data Leakage) : Supprime les colonnes qui donnent la réponse à l'avance ou qui n'existent qu'après le feu (ex: CONT_DATE date de contrôle du feu). C'est crucial pour que le modèle soit réaliste.

Imputation : Remplit les trous dans les colonnes numériques avec la Médiane.

Temps : Convertit DISCOVERY_TIME en un chiffre décimal (ex: 14h30 devient 14.5).

Étape 5 : Feature Engineering Avancé (add_advanced_features)

Elle crée de nouvelles informations basées sur la physique du feu.

Interactions : Elle multiplie le Vent par la Sécheresse (Wind_x_Dryness). Pour un feu, le vent n'est dangereux que si c'est sec.

Clustering Spatial (KMeans) : Elle utilise la Latitude et la Longitude pour grouper les feux en 50 "régions" homogènes. Cela permet au modèle de comprendre "Ah, on est dans la zone Sierra Nevada" sans avoir besoin de cartes complexes.

Étape 6 : Encodage Final (encode_final_categories)
Action : Applique le One-Hot Encoding. Elle transforme les colonnes de texte restantes (ex: "État", "Cause du feu", "Cluster Régional") en colonnes binaires (0 ou 1).

XGBoost ne lit que des chiffres. Il ne peut pas lire "Californie", mais il comprend très bien une colonne State_CA qui vaut 1.

Résumé : Ce dataset est prêt pour XGBoost !
À la sortie de full_data_cleaning_pipeline, votre DataFrame (df_final) possède les qualités suivantes :

100% Numérique : Tout le texte a été encodé ou supprimé.

Sans Fuite (Leakage Free) : Les variables du futur (post-incendie) sont retirées.

Dense : Il n'y a plus de valeurs manquantes (imputées par médiane ou indicateur).

Enrichi : Il contient des signaux forts (Saisonnalité, Interactions Vent/Sécheresse, Clusters géographiques) que XGBoost pourra exploiter pour maximiser sa performance.

In [3]:
from cleaning import *
df_clean=full_data_cleaning_pipeline(df)

Pipeline de Nettoyage - Démarrage. Dimensions : (73221, 308)
Colonnes avec plus de 50.0% de NaN à supprimer : 23
Colonnes constantes à supprimer : 8
Total de colonnes supprimées (Step 1) : 24
--- DÉBUT DU NETTOYAGE DÉTAILLÉ (handle_aberrant_values) ---
   -> 225 colonnes float vérifiées.
   -> 92 valeurs techniques (-1e30) remplacées par NaN.
   -> 3409 codes d'erreur (< -900) remplacés par NaN.
   -> 1474 erreurs topo (32767) remplacées par NaN.
Total des cellules corrigées : 4975
-> Features temporelles (Mois, Jour) extraites.
-> Conversion des colonnes 'faux-textes' en numériques...
-> Colonnes catégorielles supprimées (Manuelles) : 15
-> Imputation des NaN catégoriels par 'Missing' sur 22 colonnes.
Dimensions avant traitement des NaN : (73221, 271)
   -> 'DISCOVERY_TIME' transformée en 'Discovery_Time_Hours'.
-> Colonnes supprimées (Leakage/IDs) : 4
-> Remplissage des valeurs manquantes (Médiane)...
   ALERTE : 10 colonnes sont entièrement vides (ex: ['EVH_1km', 'EVT_1km', 'EVC_1km

Le code suivant prépare et entraine le data set de manière suivante en utisant le XGboost comme modèle: 
1. Préparation des Données (prepare_data)
Cette étape transforme les données brutes en un format utilisable par le modèle.

Simplification des Classes (Target Engineering) : Le code convertit les classes de feux d'origine (de A à G) en 3 catégories simplifiées pour faciliter la prédiction :

0 (Petit) : Classes A et B.

1 (Moyen) : Classe C.

2 (Grand/Danger) : Classes D, E, F, G.

Nettoyage : Il nettoie les noms de colonnes (suppression des caractères spéciaux) et ne garde que les caractéristiques (SELECTED_FEATURES) jugées pertinentes (ex: météo, végétation, population, état).

2. Stratégie d'Entraînement : Le "Soft Undersampling"
Les grands feux sont rares (données déséquilibrées). Si on entraîne le modèle sur les données brutes, il ignorera les grands feux.

Split : Séparation classique 80% entraînement / 20% test.

Rééquilibrage (Ratio 4:2:1) : Le code force une distribution artificielle dans les données d'entraînement:

Pour 1 Grand feu (Classe 2)...

Il garde 2 Feux Moyens (Classe 1)...

Et 4 Petits Feux (Classe 0).

3. Modélisation (XGBoost)
Le code utilise XGBoost Classifier

Paramètres clés :

objective='multi:softprob' : Le modèle ne donne pas juste une classe, mais la probabilité d'appartenir à chaque classe (ex: 10% Petit, 30% Moyen, 60% Grand).

n_estimators=500 & learning_rate=0.03 : Une configuration robuste qui crée beaucoup d'arbres (500) mais apprend lentement (0.03) pour éviter le sur-apprentissage (overfitting).

4. Évaluation et Analyse de Risque
Une fois le modèle entraîné, le code ne se contente pas de calculer la précision globale (qui serait trompeuse), il se concentre sur la détection du risque.

A. Score de Risque
Il isole la probabilité que le feu soit de Classe 2 (Grand Feu). C'est ce score (Score_Risque_GrandFeu) qui servira d'alerte.

B. Métriques Clés Affichées
AUC (Area Under Curve) : Une note globale sur 1.000. Elle mesure la capacité du modèle à classer un grand feu "plus haut" qu'un petit feu.

Fiabilité (Calibration) : Le tableau "Tranche de Risque" vérifie la cohérence.

Idéalement : Si le modèle prédit un risque de 80-100%, on veut voir que dans la réalité, ~80-100% de ces feux étaient effectivement des grands feux.

Seuil de 50% (Analyse Décisionnelle) :

Le code regarde ce qu'il se passe si on déclenche une alerte "DANGER" dès que la probabilité dépasse 50%.

Il calcule combien de Grands Feux sont correctement détectés (Recall) et combien de Petits Feux déclenchent une fausse alerte.

In [3]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.utils import resample

# ==============================================================================
# 1. CONFIGURATION (Les 50 Variables "Élues")
# ==============================================================================
SELECTED_FEATURES = [
    'LONGITUDE', 'EVH', 'EVC', 'Population', 'Popo_1km', 'GDP', 'EBLR_PFS', 
    'No_FireStation_5.0km', 'GHM', 'Ecoregion_US_L3CODE', 'Ecoregion_NA_L2CODE', 
    'Ecoregion_NA_L1CODE', 'Wind_x_Dryness', 'NWCG_REPORTING_AGENCY_FWS', 
    'NWCG_CAUSE_CLASSIFICATION_Natural', 'NWCG_GENERAL_CAUSE_Natural', 
    'NWCG_GENERAL_CAUSE_Recreation and ceremony', 'OWNER_DESCR_MUNICIPAL/LOCAL', 
    'OWNER_DESCR_PRIVATE', 'OWNER_DESCR_UNDEFINED FEDERAL', 'STATE_AL', 
    'STATE_AZ', 'STATE_KS', 'STATE_KY', 'STATE_NC', 'STATE_ND', 'STATE_OK', 
    'STATE_SC', 'STATE_UT', 'STATE_WA', 'Mang_Type_TRIB', 'Mang_Type_UNK', 
    'Des_Tp_NF', 'NAME_Blue Mountains', 'NAME_Dissected Appalachian Plateau', 
    'NAME_Flint Hills', 'NAME_High Lava Plateau and Semiarid Hills', 
    'NAME_Middle Rockies (Townsend-Elkhorn)', 'NAME_Northern Cross Timbers and Lower Canadian Hills', 
    'NAME_Snake River Plain', 'NAME_South Central California Foothills and Coastal Mountains', 
    'NAME_Western Corn-Belt and Central Irregular Plains', 'fm100_Percentile_70-90%', 
    'vpd_Percentile_5-10%', 'Region_Cluster_2', 'Region_Cluster_3', 
    'Region_Cluster_12', 'Region_Cluster_28', 'Region_Cluster_41', 'Region_Cluster_44'
]

# ==============================================================================
# 2. PRÉPARATION DES DONNÉES
# ==============================================================================
print("--- PRÉPARATION FINALE AVANT MODÉLISATION ---")

# Copie pour ne pas écraser l'original
df_model = df_clean.copy()

# 1. Mapping Target
mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}
if 'FIRE_SIZE_CLASS' in df_model.columns:
    if df_model['FIRE_SIZE_CLASS'].dtype == 'object':
        df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(str).map(mapping)
    df_model = df_model.dropna(subset=['FIRE_SIZE_CLASS'])
    df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(int)

# 2. Nettoyage noms colonnes (Regex XGBoost)
new_cols = {c: re.sub(r'[\[\]<]', '_', c).strip() for c in df_model.columns}
df_model = df_model.rename(columns=new_cols)

# 3. Filtrage
cols_to_keep = SELECTED_FEATURES + ['FIRE_SIZE_CLASS']
target_series = df_model['FIRE_SIZE_CLASS']
df_optimized = df_model.reindex(columns=cols_to_keep, fill_value=0)
df_optimized['FIRE_SIZE_CLASS'] = target_series

print(f"-> Données prêtes. Dimensions : {df_optimized.shape}")

# ==============================================================================
# 3. SPLIT & ENTRAÎNEMENT (Soft Undersampling)
# ==============================================================================
X = df_optimized.drop(columns=['FIRE_SIZE_CLASS'])
y = df_optimized['FIRE_SIZE_CLASS']

print("\n--- ENTRAÎNEMENT DU MODÈLE XGBOOST ---")
X_train_raw, X_test, y_train_raw, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Équilibrage 4:2:1
train_data = pd.concat([X_train_raw, y_train_raw], axis=1)
df_0 = train_data[train_data.FIRE_SIZE_CLASS == 0]
df_1 = train_data[train_data.FIRE_SIZE_CLASS == 1]
df_2 = train_data[train_data.FIRE_SIZE_CLASS == 2]

n_minority = len(df_2)
df_0_down = resample(df_0, replace=False, n_samples=min(len(df_0), n_minority * 4), random_state=42)
df_1_down = resample(df_1, replace=False, n_samples=min(len(df_1), n_minority * 2), random_state=42)
df_train_balanced = pd.concat([df_0_down, df_1_down, df_2]).sample(frac=1, random_state=42)

X_train = df_train_balanced.drop(columns=['FIRE_SIZE_CLASS'])
y_train = df_train_balanced['FIRE_SIZE_CLASS']

model = xgb.XGBClassifier(
    objective='multi:softprob', 
    num_class=3,
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

model.fit(X_train, y_train)
print("-> Modèle entraîné avec succès.")

# ==============================================================================
# 4. PRÉDICTIONS ET STOCKAGE COMPLET
# ==============================================================================
print("\n" + "="*80)
print(" CALCUL DES RÉSULTATS DÉTAILLÉS")
print("="*80)

probas = model.predict_proba(X_test)

results = X_test.copy()
results['True_Class'] = y_test            
results['Prob_Class_0'] = probas[:, 0]    
results['Prob_Class_1'] = probas[:, 1]    
results['Prob_Class_2'] = probas[:, 2]    

# Calculs additionnels
results['Score_Risque_GrandFeu'] = results['Prob_Class_2']
results['Score_Risque_Percent'] = (results['Score_Risque_GrandFeu'] * 100).round(1)

# A. Score Global (AUC)
y_test_binary = (y_test == 2).astype(int) 
auc = roc_auc_score(y_test_binary, results['Prob_Class_2'])
print(f"SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : {auc:.3f}")
print("-" * 80)



# B. Analyse par seuil de 50%
print("\nANALYSE DE RISQUE (Seuil > 50%)")
for cl, label in [(0, "Petits Feux"), (1, "Moyens Feux"), (2, "Grands Feux")]:
    # Correction ici : utilisation de 'True_Class' au lieu de 'Vrai_Classe'
    subset = results[results['True_Class'] == cl]
    nb_alert = (subset['Score_Risque_GrandFeu'] > 0.5).sum()
    pct_alert = (nb_alert / len(subset)) * 100 if len(subset) > 0 else 0
    print(f"{label:<12} détectés à haut risque : {pct_alert:6.2f}% ({nb_alert}/{len(subset)})")
print("-" * 80)

--- PRÉPARATION FINALE AVANT MODÉLISATION ---
-> Données prêtes. Dimensions : (73221, 51)

--- ENTRAÎNEMENT DU MODÈLE XGBOOST ---
-> Modèle entraîné avec succès.

 CALCUL DES RÉSULTATS DÉTAILLÉS
SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : 0.902
--------------------------------------------------------------------------------

ANALYSE DE RISQUE (Seuil > 50%)
Petits Feux  détectés à haut risque :   1.34% (175/13062)
Moyens Feux  détectés à haut risque :   6.76% (79/1169)
Grands Feux  détectés à haut risque :  32.85% (136/414)
--------------------------------------------------------------------------------


Ici on à une très bonne compréhension du data set avec une AUC de 0.902 c'est exceptionnel cependant malgrès cela, nos résultats ne sont pas tant utile à nos pompiers car notre modèle ne detecte que très peu les grands feu, il faut donc changer quelque chose

In [4]:
def print_calibration_table_notebook(df_results, class_id, class_name):
    """
    Affiche la table de fiabilité pour une classe donnée avec le formatage spécifique demandé.
    """
    col_score = f'Prob_Class_{class_id}'
    
    # 1. Création des tranches de probabilités (bins)
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    
    # On travaille sur une copie pour ne pas modifier l'original
    df_temp = df_results.copy()
    df_temp['Tranche'] = pd.cut(df_temp[col_score], bins=bins, labels=labels, include_lowest=True)

    # 2. Affichage de l'en-tête
    print(f"\n>>> ANALYSE DE FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    # 3. Calculs et affichage ligne par ligne
    # observed=False permet d'afficher même les tranches vides (si catégories pandas)
    grouped = df_temp.groupby('Tranche', observed=False)
    
    for label in labels:
        if label in grouped.groups:
            group = grouped.get_group(label)
            count = len(group)
            if count > 0:
                # Calcul du pourcentage de Vrais Positifs dans cette tranche
                real_percentage = (group['True_Class'] == class_id).mean() * 100
                print(f"{label:<20} | {count:<10} | {real_percentage:.1f}%")
            else:
                 print(f"{label:<20} | 0          | -")
        else:
            print(f"{label:<20} | 0          | -")
            
    print("-" * 65)

# --- EXÉCUTION DE L'ANALYSE ---

# S'assurer que 'results' est bien défini (issu du bloc précédent)
if 'results' in locals():
    print_calibration_table_notebook(results, 0, "Petit Feu")
    print_calibration_table_notebook(results, 1, "Moyen Feu")
    print_calibration_table_notebook(results, 2, "Grand Feu")
else:
    print("Erreur : Le DataFrame 'results' est introuvable. Veuillez exécuter la cellule d'entraînement précédente.")
from sklearn.metrics import recall_score
y_pred = model.predict(X_test)
recall_scores = recall_score(y_test, y_pred, average=None, zero_division=0)
recall_grand_feu = recall_scores[2]

print(f"Rappels par classe (0, 1, 2) : {recall_scores}")
print(f"RECALL DES GRANDS FEUX (Classe 2) : {recall_grand_feu:.2%}")


>>> ANALYSE DE FIABILITÉ : PETIT FEU (Classe 0)
Tranche de Proba     | Nb Feux    | % Réellement Petit Feu
-----------------------------------------------------------------
0-20%                | 1001       | 49.7%
20-40%               | 1706       | 72.1%
40-60%               | 1887       | 84.7%
60-80%               | 2791       | 93.1%
80-100%              | 7260       | 98.3%
-----------------------------------------------------------------

>>> ANALYSE DE FIABILITÉ : MOYEN FEU (Classe 1)
Tranche de Proba     | Nb Feux    | % Réellement Moyen Feu
-----------------------------------------------------------------
0-20%                | 8625       | 2.1%
20-40%               | 3136       | 9.9%
40-60%               | 1996       | 20.5%
60-80%               | 862        | 30.3%
80-100%              | 26         | 38.5%
-----------------------------------------------------------------

>>> ANALYSE DE FIABILITÉ : GRAND FEU (Classe 2)
Tranche de Proba     | Nb Feux    | % Réellement Gran

Les resultats ici confirme notre première analyse, on à un recall des grands feux de seulement 43%

Nous allons essayer d'utiliser un modèle beaucoup plus agressif en utilisant des poids de réequilibrage plus poussé : [1, 5, 20]

In [5]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score

# ==============================================================================
# 1. CONFIGURATION (Les 50 Variables "Élues")
# ==============================================================================
SELECTED_FEATURES = [
    'LONGITUDE', 'EVH', 'EVC', 'Population', 'Popo_1km', 'GDP', 'EBLR_PFS', 
    'No_FireStation_5.0km', 'GHM', 'Ecoregion_US_L3CODE', 'Ecoregion_NA_L2CODE', 
    'Ecoregion_NA_L1CODE', 'Wind_x_Dryness', 'NWCG_REPORTING_AGENCY_FWS', 
    'NWCG_CAUSE_CLASSIFICATION_Natural', 'NWCG_GENERAL_CAUSE_Natural', 
    'NWCG_GENERAL_CAUSE_Recreation and ceremony', 'OWNER_DESCR_MUNICIPAL/LOCAL', 
    'OWNER_DESCR_PRIVATE', 'OWNER_DESCR_UNDEFINED FEDERAL', 'STATE_AL', 
    'STATE_AZ', 'STATE_KS', 'STATE_KY', 'STATE_NC', 'STATE_ND', 'STATE_OK', 
    'STATE_SC', 'STATE_UT', 'STATE_WA', 'Mang_Type_TRIB', 'Mang_Type_UNK', 
    'Des_Tp_NF', 'NAME_Blue Mountains', 'NAME_Dissected Appalachian Plateau', 
    'NAME_Flint Hills', 'NAME_High Lava Plateau and Semiarid Hills', 
    'NAME_Middle Rockies (Townsend-Elkhorn)', 'NAME_Northern Cross Timbers and Lower Canadian Hills', 
    'NAME_Snake River Plain', 'NAME_South Central California Foothills and Coastal Mountains', 
    'NAME_Western Corn-Belt and Central Irregular Plains', 'fm100_Percentile_70-90%', 
    'vpd_Percentile_5-10%', 'Region_Cluster_2', 'Region_Cluster_3', 
    'Region_Cluster_12', 'Region_Cluster_28', 'Region_Cluster_41', 'Region_Cluster_44'
]

# ==============================================================================
# 2. PRÉPARATION DES DONNÉES
# ==============================================================================
print("--- PRÉPARATION FINALE AVANT MODÉLISATION (MODE AGRESSIF) ---")

# Copie de sécurité
df_model = df_clean.copy()

# 1. Mapping Target
mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}
if 'FIRE_SIZE_CLASS' in df_model.columns:
    if df_model['FIRE_SIZE_CLASS'].dtype == 'object':
        df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(str).map(mapping)
    df_model = df_model.dropna(subset=['FIRE_SIZE_CLASS'])
    df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(int)

# 2. Nettoyage noms colonnes (Regex XGBoost)
new_cols = {c: re.sub(r'[\[\]<]', '_', c).strip() for c in df_model.columns}
df_model = df_model.rename(columns=new_cols)

# 3. Filtrage
cols_to_keep = SELECTED_FEATURES + ['FIRE_SIZE_CLASS']
target_series = df_model['FIRE_SIZE_CLASS']
df_optimized = df_model.reindex(columns=cols_to_keep, fill_value=0)
df_optimized['FIRE_SIZE_CLASS'] = target_series

print(f"-> Données prêtes. Dimensions : {df_optimized.shape}")

# ==============================================================================
# 3. SPLIT & ENTRAÎNEMENT (Pondération 1:5:20)
# ==============================================================================
X = df_optimized.drop(columns=['FIRE_SIZE_CLASS'])
y = df_optimized['FIRE_SIZE_CLASS']

print("\n--- ENTRAÎNEMENT DU MODÈLE XGBOOST (COST-SENSITIVE) ---")

# Split classique (sans resampling, car on utilise des poids)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# CRÉATION DES POIDS [1, 5, 20]
# 0 (Petit) = Poids 1
# 1 (Moyen) = Poids 5
# 2 (Grand) = Poids 20
weights_map = {0: 1, 1: 5, 2: 20}
sample_weights = y_train.map(weights_map)

print(f"-> Poids appliqués : {weights_map}")
print(f"-> Exemple de poids (10 premiers) : {sample_weights.head(10).values}")

model = xgb.XGBClassifier(
    objective='multi:softprob', 
    num_class=3,
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

# Entraînement avec sample_weight
model.fit(X_train, y_train, sample_weight=sample_weights)
print("-> Modèle entraîné avec pondération agressive.")

# ==============================================================================
# 4. PRÉDICTIONS ET STOCKAGE COMPLET
# ==============================================================================
print("\n" + "="*80)
print(" CALCUL DES RÉSULTATS DÉTAILLÉS (APPROCHE AGRESSIVE)")
print("="*80)

probas = model.predict_proba(X_test)

results = X_test.copy()
results['True_Class'] = y_test            
results['Prob_Class_0'] = probas[:, 0]    
results['Prob_Class_1'] = probas[:, 1]    
results['Prob_Class_2'] = probas[:, 2]    

# Calculs additionnels
results['Score_Risque_GrandFeu'] = results['Prob_Class_2']
results['Score_Risque_Percent'] = (results['Score_Risque_GrandFeu'] * 100).round(1)

# A. Score Global (AUC)
y_test_binary = (y_test == 2).astype(int) 
auc = roc_auc_score(y_test_binary, results['Prob_Class_2'])
print(f"SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : {auc:.3f}")
print("-" * 80)

# B. Analyse par seuil de 50%
print("\nANALYSE DE RISQUE (Seuil > 50%)")
for cl, label in [(0, "Petits Feux"), (1, "Moyens Feux"), (2, "Grands Feux")]:
    subset = results[results['True_Class'] == cl]
    nb_alert = (subset['Score_Risque_GrandFeu'] > 0.5).sum()
    pct_alert = (nb_alert / len(subset)) * 100 if len(subset) > 0 else 0
    print(f"{label:<12} détectés à haut risque : {pct_alert:6.2f}% ({nb_alert}/{len(subset)})")
print("-" * 80)

# ==============================================================================
# 5. FONCTION D'ANALYSE DE CALIBRATION
# ==============================================================================
def print_calibration_table_notebook(df_results, class_id, class_name):
    """
    Affiche la table de fiabilité pour une classe donnée.
    """
    col_score = f'Prob_Class_{class_id}'
    
    # 1. Création des tranches de probabilités (bins)
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    
    # Copie locale
    df_temp = df_results.copy()
    df_temp['Tranche'] = pd.cut(df_temp[col_score], bins=bins, labels=labels, include_lowest=True)

    # 2. Affichage de l'en-tête
    print(f"\n>>> ANALYSE DE FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    # 3. Calculs et affichage ligne par ligne
    grouped = df_temp.groupby('Tranche', observed=False)
    
    for label in labels:
        if label in grouped.groups:
            group = grouped.get_group(label)
            count = len(group)
            if count > 0:
                # Calcul du pourcentage de Vrais Positifs dans cette tranche
                real_percentage = (group['True_Class'] == class_id).mean() * 100
                print(f"{label:<20} | {count:<10} | {real_percentage:.1f}%")
            else:
                 print(f"{label:<20} | 0          | -")
        else:
            print(f"{label:<20} | 0          | -")
            
    print("-" * 65)



--- PRÉPARATION FINALE AVANT MODÉLISATION (MODE AGRESSIF) ---
-> Données prêtes. Dimensions : (73221, 51)

--- ENTRAÎNEMENT DU MODÈLE XGBOOST (COST-SENSITIVE) ---
-> Poids appliqués : {0: 1, 1: 5, 2: 20}
-> Exemple de poids (10 premiers) : [ 1  5 20  1  1  1  1  1  1  1]
-> Modèle entraîné avec pondération agressive.

 CALCUL DES RÉSULTATS DÉTAILLÉS (APPROCHE AGRESSIVE)
SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : 0.901
--------------------------------------------------------------------------------

ANALYSE DE RISQUE (Seuil > 50%)
Petits Feux  détectés à haut risque :   3.47% (453/13062)
Moyens Feux  détectés à haut risque :  17.11% (200/1169)
Grands Feux  détectés à haut risque :  51.21% (212/414)
--------------------------------------------------------------------------------


In [6]:
import pandas as pd
import numpy as np
from sklearn.metrics import recall_score

# ==============================================================================
# FONCTION D'ANALYSE DÉTAILLÉE (AVEC RÉPARTITION RÉELLE)
# ==============================================================================
def afficher_analyse_fiabilite_detaillee(df, class_id, nom_classe):
    """
    Analyse la fiabilité par tranche de probabilité et montre 
    la répartition réelle des classes (0, 1, 2) dans chaque tranche.
    """
    col_proba = f'Prob_Class_{class_id}'
    
    # Bins de probabilité
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    
    # Copie de travail
    df_work = df.copy()
    df_work['Tranche'] = pd.cut(df_work[col_proba], bins=bins, labels=labels, include_lowest=True)
    
    # En-tête du tableau
    print(f"\n>>> ANALYSE DÉTAILLÉE : {nom_classe.upper()} (Classe {class_id})")
    # On prépare un header plus large pour voir la répartition
    print(f"{'Tranche':<10} | {'Nb Feux':<8} || {'% Vrai 0':<9} | {'% Vrai 1':<9} | {'% Vrai 2':<9}")
    print(f"{'(Proba)':<10} | {'(Total)':<8} || {'(Petit)':<9} | {'(Moyen)':<9} | {'(Grand)':<9}")
    print("-" * 65)
    
    # Boucle de filtrage direct (Robuste)
    for label in labels:
        subset = df_work[df_work['Tranche'] == label]
        count = len(subset)
        
        if count > 0:
            # Calcul de la répartition réelle dans cette tranche
            pct_0 = (subset['True_Class'] == 0).mean() * 100
            pct_1 = (subset['True_Class'] == 1).mean() * 100
            pct_2 = (subset['True_Class'] == 2).mean() * 100
            
            # Mise en évidence de la classe cible avec une *
            # (optionnel, mais aide à la lecture)
            print(f"{label:<10} | {count:<8} || {pct_0:7.1f}%  | {pct_1:7.1f}%  | {pct_2:7.1f}%")
        else:
            print(f"{label:<10} | 0        ||    -      |    -      |    -     ")
            
    print("-" * 65)

# ==============================================================================
# EXÉCUTION
# ==============================================================================

# On lance l'analyse détaillée
# Cela répond à votre question : "Quand le modèle est sûr à 80% que c'est un Grand Feu,
# est-ce que les erreurs sont des Moyens ou des Petits ?"

afficher_analyse_fiabilite_detaillee(results, 0, "Petit Feu")
afficher_analyse_fiabilite_detaillee(results, 1, "Moyen Feu")
afficher_analyse_fiabilite_detaillee(results, 2, "Grand Feu")

# ==============================================================================
# RAPPEL DU RECALL GLOBAL
# ==============================================================================
y_true = results['True_Class']
y_pred = results[['Prob_Class_0', 'Prob_Class_1', 'Prob_Class_2']].idxmax(axis=1).str.extract(r'(\d+)').astype(int)[0]
recalls = recall_score(y_true, y_pred, average=None)

print(f"\nRAPPEL (RECALL) GLOBAL :")
print(f"Petit : {recalls[0]:.2%} | Moyen : {recalls[1]:.2%} | Grand : {recalls[2]:.2%}")


>>> ANALYSE DÉTAILLÉE : PETIT FEU (Classe 0)
Tranche    | Nb Feux  || % Vrai 0  | % Vrai 1  | % Vrai 2 
(Proba)    | (Total)  || (Petit)   | (Moyen)   | (Grand)  
-----------------------------------------------------------------
0-20%      | 1025     ||    48.5%  |    30.4%  |    21.1%
20-40%     | 1629     ||    71.6%  |    23.0%  |     5.4%
40-60%     | 1918     ||    85.4%  |    10.8%  |     3.8%
60-80%     | 2663     ||    93.0%  |     6.1%  |     0.9%
80-100%    | 7410     ||    98.3%  |     1.5%  |     0.2%
-----------------------------------------------------------------

>>> ANALYSE DÉTAILLÉE : MOYEN FEU (Classe 1)
Tranche    | Nb Feux  || % Vrai 0  | % Vrai 1  | % Vrai 2 
(Proba)    | (Total)  || (Petit)   | (Moyen)   | (Grand)  
-----------------------------------------------------------------
0-20%      | 10073    ||    94.9%  |     3.1%  |     2.0%
20-40%     | 2753     ||    80.6%  |    14.1%  |     5.3%
40-60%     | 1538     ||    72.6%  |    24.1%  |     3.4%
60-80%    

Nous avons ici un bien meilleur résultat, qui nous permet d'avoir un bon pourcentage de recall mais avec cependant peut être un peu trop d'alerte sur des feu, nous allons maintenant essaye un modèle très agressif xgboost

In [4]:
import pandas as pd
import numpy as np
import xgboost as xgb
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score

# ==============================================================================
# 0. FONCTION D'ANALYSE (Définie au début pour être disponible)
# ==============================================================================
def print_calibration_table_notebook(df_results, class_id, class_name):
    """
    Affiche la table de fiabilité pour une classe donnée.
    """
    col_score = f'Prob_Class_{class_id}'
    
    # Création des tranches de probabilités (bins)
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    
    # Copie locale
    df_temp = df_results.copy()
    df_temp['Tranche'] = pd.cut(df_temp[col_score], bins=bins, labels=labels, include_lowest=True)

    print(f"\n>>> ANALYSE DE FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    grouped = df_temp.groupby('Tranche', observed=False)
    
    for label in labels:
        if label in grouped.groups:
            group = grouped.get_group(label)
            count = len(group)
            if count > 0:
                real_percentage = (group['True_Class'] == class_id).mean() * 100
                print(f"{label:<20} | {count:<10} | {real_percentage:.1f}%")
            else:
                 print(f"{label:<20} | 0          | -")
        else:
            print(f"{label:<20} | 0          | -")
    print("-" * 65)

# ==============================================================================
# 1. CONFIGURATION (Les 50 Variables "Élues")
# ==============================================================================
SELECTED_FEATURES = [
    'LONGITUDE', 'EVH', 'EVC', 'Population', 'Popo_1km', 'GDP', 'EBLR_PFS', 
    'No_FireStation_5.0km', 'GHM', 'Ecoregion_US_L3CODE', 'Ecoregion_NA_L2CODE', 
    'Ecoregion_NA_L1CODE', 'Wind_x_Dryness', 'NWCG_REPORTING_AGENCY_FWS', 
    'NWCG_CAUSE_CLASSIFICATION_Natural', 'NWCG_GENERAL_CAUSE_Natural', 
    'NWCG_GENERAL_CAUSE_Recreation and ceremony', 'OWNER_DESCR_MUNICIPAL/LOCAL', 
    'OWNER_DESCR_PRIVATE', 'OWNER_DESCR_UNDEFINED FEDERAL', 'STATE_AL', 
    'STATE_AZ', 'STATE_KS', 'STATE_KY', 'STATE_NC', 'STATE_ND', 'STATE_OK', 
    'STATE_SC', 'STATE_UT', 'STATE_WA', 'Mang_Type_TRIB', 'Mang_Type_UNK', 
    'Des_Tp_NF', 'NAME_Blue Mountains', 'NAME_Dissected Appalachian Plateau', 
    'NAME_Flint Hills', 'NAME_High Lava Plateau and Semiarid Hills', 
    'NAME_Middle Rockies (Townsend-Elkhorn)', 'NAME_Northern Cross Timbers and Lower Canadian Hills', 
    'NAME_Snake River Plain', 'NAME_South Central California Foothills and Coastal Mountains', 
    'NAME_Western Corn-Belt and Central Irregular Plains', 'fm100_Percentile_70-90%', 
    'vpd_Percentile_5-10%', 'Region_Cluster_2', 'Region_Cluster_3', 
    'Region_Cluster_12', 'Region_Cluster_28', 'Region_Cluster_41', 'Region_Cluster_44'
]

# ==============================================================================
# 2. PRÉPARATION DES DONNÉES
# ==============================================================================
print("--- PRÉPARATION FINALE AVANT MODÉLISATION (MODE OPTIMISÉ) ---")

# Copie de sécurité (Assure-toi que df_clean est chargé en mémoire)
df_model = df_clean.copy()

# 1. Mapping Target
mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}
if 'FIRE_SIZE_CLASS' in df_model.columns:
    if df_model['FIRE_SIZE_CLASS'].dtype == 'object':
        df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(str).map(mapping)
    df_model = df_model.dropna(subset=['FIRE_SIZE_CLASS'])
    df_model['FIRE_SIZE_CLASS'] = df_model['FIRE_SIZE_CLASS'].astype(int)

# 2. Nettoyage noms colonnes (Regex XGBoost)
new_cols = {c: re.sub(r'[\[\]<]', '_', c).strip() for c in df_model.columns}
df_model = df_model.rename(columns=new_cols)

# 3. Filtrage
cols_to_keep = SELECTED_FEATURES + ['FIRE_SIZE_CLASS']
target_series = df_model['FIRE_SIZE_CLASS']
df_optimized = df_model.reindex(columns=cols_to_keep, fill_value=0)
df_optimized['FIRE_SIZE_CLASS'] = target_series

print(f"-> Données prêtes. Dimensions : {df_optimized.shape}")

# ==============================================================================
# 3. SPLIT & ENTRAÎNEMENT (Pondération 1:10:50 & Hyperparamètres)
# ==============================================================================
X = df_optimized.drop(columns=['FIRE_SIZE_CLASS'])
y = df_optimized['FIRE_SIZE_CLASS']

print("\n--- ENTRAÎNEMENT DU MODÈLE XGBOOST (PARAMS OPTIMISÉS) ---")

# Split classique
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# A. CRÉATION DES POIDS (Nouveaux poids : 1, 10, 50)
weights_map = {0: 1, 1: 10, 2: 50}
sample_weights = y_train.map(weights_map)

print(f"-> Poids appliqués : {weights_map}")

# B. DÉFINITION DES PARAMÈTRES OPTIMISÉS
params = {
    'n_estimators': 437,
    'max_depth': 5,
    'learning_rate': 0.08072903679293161,
    'min_child_weight': 1,
    'subsample': 0.8489574811277885,
    'colsample_bytree': 0.797160873700788,
    'reg_lambda': 0.1572046494326245,
    'reg_alpha': 0.03137111664135713,
    'objective': 'multi:softprob',
    'num_class': 3,
    'random_state': 42,
    'n_jobs': -1
}

# C. INITIALISATION ET ENTRAÎNEMENT
# On utilise **params pour unpacker le dictionnaire directement
model = xgb.XGBClassifier(**params)

model.fit(X_train, y_train, sample_weight=sample_weights)
print("-> Modèle entraîné avec succès (Hyperparamètres chargés).")

# ==============================================================================
# 4. PRÉDICTIONS ET RÉSULTATS DÉTAILLÉS
# ==============================================================================
print("\n" + "="*80)
print(" CALCUL DES RÉSULTATS DÉTAILLÉS (APPROCHE 1:10:50)")
print("="*80)

probas = model.predict_proba(X_test)

results = X_test.copy()
results['True_Class'] = y_test            
results['Prob_Class_0'] = probas[:, 0]    
results['Prob_Class_1'] = probas[:, 1]    
results['Prob_Class_2'] = probas[:, 2]    

# Calculs additionnels
results['Score_Risque_GrandFeu'] = results['Prob_Class_2']
results['Score_Risque_Percent'] = (results['Score_Risque_GrandFeu'] * 100).round(1)

# A. Score Global (AUC)
y_test_binary = (y_test == 2).astype(int) 
auc = roc_auc_score(y_test_binary, results['Prob_Class_2'])
print(f"SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : {auc:.3f}")
print("-" * 80)

# B. Analyse par seuil de 50%
print("\nANALYSE DE RISQUE (Seuil > 50%)")
for cl, label in [(0, "Petits Feux"), (1, "Moyens Feux"), (2, "Grands Feux")]:
    subset = results[results['True_Class'] == cl]
    nb_alert = (subset['Score_Risque_GrandFeu'] > 0.5).sum()
    pct_alert = (nb_alert / len(subset)) * 100 if len(subset) > 0 else 0
    print(f"{label:<12} détectés à haut risque : {pct_alert:6.2f}% ({nb_alert}/{len(subset)})")
print("-" * 80)

# C. Table de calibration pour les Grands Feux
print_calibration_table_notebook(results, class_id=2, class_name="Grands Feux")

--- PRÉPARATION FINALE AVANT MODÉLISATION (MODE OPTIMISÉ) ---
-> Données prêtes. Dimensions : (73221, 51)

--- ENTRAÎNEMENT DU MODÈLE XGBOOST (PARAMS OPTIMISÉS) ---
-> Poids appliqués : {0: 1, 1: 10, 2: 50}
-> Modèle entraîné avec succès (Hyperparamètres chargés).

 CALCUL DES RÉSULTATS DÉTAILLÉS (APPROCHE 1:10:50)
SCORE DE PERFORMANCE GLOBAL (AUC Grand Feu) : 0.893
--------------------------------------------------------------------------------

ANALYSE DE RISQUE (Seuil > 50%)
Petits Feux  détectés à haut risque :   5.69% (743/13062)
Moyens Feux  détectés à haut risque :  21.64% (253/1169)
Grands Feux  détectés à haut risque :  60.39% (250/414)
--------------------------------------------------------------------------------

>>> ANALYSE DE FIABILITÉ : GRANDS FEUX (Classe 2)
Tranche de Proba     | Nb Feux    | % Réellement Grands Feux
-----------------------------------------------------------------
0-20%                | 11007      | 0.6%
20-40%               | 1822       | 3.5%
40-60

Le resultat que nous avons ici est très bon car il a une auc de 0.893 ce qui est excellent tout en détectant bien les grands feu sans avoir beaucoup de fausse alerte.

Nous allons maintenant essayé de tout refaire avec le catboost, cela necessite que nous modifions notre cleaning pour garder les columns catégorielle sans les encoder avec notre fichier catclean

In [7]:
from catclean import * 
df_catboost=without_encode(df)

Pipeline de Nettoyage (Mode CatBoost) - Démarrage. Dimensions : (73221, 308)
Colonnes avec plus de 50.0% de NaN à supprimer : 23
Colonnes constantes à supprimer : 8
Total de colonnes supprimées (Step 1) : 24
--- DÉBUT DU NETTOYAGE DÉTAILLÉ (handle_aberrant_values) ---
   -> 225 colonnes float vérifiées.
   -> 92 valeurs techniques (-1e30) remplacées par NaN.
   -> 3409 codes d'erreur (< -900) remplacés par NaN.
   -> 1474 erreurs topo (32767) remplacées par NaN.
Total des cellules corrigées : 4975
-> Features temporelles (Mois, Jour) extraites.
-> Conversion des colonnes 'faux-textes' en numériques...
-> Colonnes catégorielles supprimées (Manuelles) : 15
-> Imputation des NaN catégoriels par 'Missing' sur 22 colonnes.
Dimensions avant traitement des NaN : (73221, 271)
   -> 'DISCOVERY_TIME' transformée en 'Discovery_Time_Hours'.
-> Colonnes supprimées (Leakage/IDs) : 4
-> Sauvetage des colonnes critiques : ['EVH_1km', 'EVT_1km', 'EVC_1km', 'CH_1km', 'Land_Cover_1km']
   -> EVH_1km : 73

appliquons le modèle et regardons ses résultats

In [ ]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, precision_score

# --- 1. CONFIGURATION (Les 50 Variables "Élues") ---
BEST_FEATURES = [
    'Popo_1km', 'STATE', 'Population', 'EVH', 'EVC', 'rpms_1km', 'OWNER_DESCR', 
    'FOD_ID', 'Discovery_Time_Hours', 'NWCG_GENERAL_CAUSE', 'Region_Cluster', 
    'EBLR_PFS', 'EALR_PFS', 'No_FireStation_20.0km', 'Wind_x_Dryness', 'Land_Cover', 
    'EVT', 'EPL_AGE17', 'LATITUDE', 'GHM', 'NWCG_CAUSE_CLASSIFICATION', 'EPL_MOBILE', 
    'Ecoregion_US_L3CODE', 'NDVI-1day', 'Ecoregion_NA_L2CODE', 'Mang_Type', 'GDP', 
    'pr_Normal', 'EPL_NOHSDP', 'PM25F_PFS', 'bi_Percentile', 'rmin', 'sph', 
    'Annual_precipitation', 'HBF_PFS', 'vpd_Normal', 'Slope_x_Wind', 'sph_Normal', 
    'Slope_1km', 'rmax_Normal', 'bi_Normal', 'TRACT', 'Elevation', 'RPL_THEME4', 
    'EPLR_PFS', 'MHVF_PFS', 'RPL_THEME2', 'SDI', 'Wind_x_Potential', 'RMP_PFS'
]

# --- 2. HYPERPARAMÈTRES DE BASE ---
BASE_PARAMS = {
    'iterations': 1000,          
    'learning_rate': 0.094012,   
    'depth': 7,                  
    'l2_leaf_reg': 7.827,        
    'bagging_temperature': 0.703,
    'random_strength': 9.302,    
    'border_count': 182,         
    'loss_function': 'MultiClass',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'allow_writing_files': False,
    'verbose': 100
}

# --- 3. FONCTION UTILITAIRE (AFFICHAGE) ---
def print_calibration_table(df_results, class_id, class_name):
    """ Affiche la table de fiabilité pour une classe donnée """
    col_score = f'Prob_Class_{class_id}'
    
    # Création des tranches de probabilités
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    df_results['Tranche'] = pd.cut(df_results[col_score], bins=bins, labels=labels)

    print(f"\n>>> ANALYSE DE FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    counts = df_results['Tranche'].value_counts(sort=False)
    
    for label in labels:
        if label in counts.index and counts[label] > 0:
            subset = df_results[df_results['Tranche'] == label]
            real_percentage = (subset['True_Class'] == class_id).mean() * 100
            print(f"{label:<20} | {counts[label]:<10} | {real_percentage:.1f}%")
        else:
            print(f"{label:<20} | 0          | -")
    print("-" * 65)

# ==============================================================================
# PRÉPARATION DES DONNÉES (Basé sur df_catboost existant)
# ==============================================================================

# On travaille sur une copie pour ne pas modifier l'original par erreur
df_working = df_catboost.copy()

print("--- 1. PRÉPARATION DES DONNÉES ---")

# Création de la Cible (Mapping 3 Classes comme dans le script original)
mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}

if 'FIRE_SIZE_CLASS' in df_working.columns:
    df_working = df_working.dropna(subset=['FIRE_SIZE_CLASS'])
    
    # Vérification si le mapping est nécessaire (si les données sont encore des lettres)
    val_sample = df_working['FIRE_SIZE_CLASS'].iloc[0]
    if isinstance(val_sample, str) and val_sample in mapping:
         df_working['FIRE_SIZE_CLASS'] = df_working['FIRE_SIZE_CLASS'].map(mapping)
    
    # Conversion en entier
    df_working['FIRE_SIZE_CLASS'] = df_working['FIRE_SIZE_CLASS'].astype(int)

# Filtrage pour ne garder que les colonnes existantes parmi les "BEST_FEATURES"
existing_features = [f for f in BEST_FEATURES if f in df_working.columns]
X = df_working[existing_features]
y = df_working['FIRE_SIZE_CLASS']

# Identification des features catégorielles
cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
print(f"Features utilisées : {len(existing_features)}")
print(f"Features catégorielles détectées : {len(cat_features)}")

# Split (Stratifié pour garder les proportions initiales)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)
print(f"Train size: {X_train.shape} | Test size: {X_test.shape}")

# ==============================================================================
# ENTRAÎNEMENT & RÉSULTATS
# ==============================================================================

# DÉFINITION DES POIDS (STRATÉGIE 4 - TRÈS AGRESSIF)
selected_weights = [1, 5, 20]
print(f"\n>>> STRATÉGIE DE POIDS APPLIQUÉE : [Petit: 1, Moyen: 5, Grand: 20]")

print("\n--- Entraînement du modèle CatBoost ---")

# Fusion des paramètres
final_params = BASE_PARAMS.copy()
final_params['class_weights'] = selected_weights

model = CatBoostClassifier(**final_params, cat_features=cat_features)

model.fit(
    X_train, y_train,
    eval_set=(X_test, y_test),
    early_stopping_rounds=50
)

# Prédictions & Scores
print("\n" + "="*90)
print(" RÉSULTATS DÉTAILLÉS MULTI-CLASSES (OPTIMISÉS SÉCURITÉ)")
print("="*90)

probas = model.predict_proba(X_test)
preds = model.predict(X_test)

# Scores
auc = roc_auc_score(y_test, probas, multi_class='ovr', average='weighted')
acc = accuracy_score(y_test, preds)
# Recall par classe (le [2] est celui du Grand Feu)
recalls = recall_score(y_test, preds, average=None, zero_division=0)

print(f"AUC Global (Weighted) : {auc:.4f}")
print(f"Accuracy Globale      : {acc:.4f}")
print(f"Recall Grand Feu      : {recalls[2]:.2%} (Priorité absolue)")

# Préparation du DataFrame d'analyse
results = X_test.copy()
results['True_Class'] = y_test
results['Predicted_Class'] = preds
results['Prob_Class_0'] = probas[:, 0] # Petit
results['Prob_Class_1'] = probas[:, 1] # Moyen
results['Prob_Class_2'] = probas[:, 2] # Grand

# --- PARTIE A : TOP 10 DES PLUS GROS RISQUES ---
print("\n--- TOP 10 ALERTES (Triées par probabilité de Grand Feu) ---")
top_alerts = results.sort_values(by='Prob_Class_2', ascending=False).head(10)

# En-tête avec les 3 probabilités
print(f"{'P(Petit)':<10} | {'P(Moyen)':<10} | {'P(Grand)':<10} | {'VRAI':<6} | {'ÉTAT':<6} | {'CAUSE'}")
print("-" * 90)

for idx, row in top_alerts.iterrows():
    p0 = f"{row['Prob_Class_0']*100:.1f}%"
    p1 = f"{row['Prob_Class_1']*100:.1f}%"
    p2 = f"{row['Prob_Class_2']*100:.1f}%"
    
    # Gestion des valeurs manquantes pour l'affichage
    cause = str(row.get('NWCG_GENERAL_CAUSE', 'N/A'))[:20]
    state = str(row.get('STATE', 'N/A'))
    true_c = str(row['True_Class'])
    
    print(f"{p0:<10} | {p1:<10} | {p2:<10} | {true_c:<6} | {state:<6} | {cause}")

# --- PARTIE B : ANALYSE DE FIABILITÉ ---
print_calibration_table(results, 0, "Petit Feu")
print_calibration_table(results, 1, "Moyen Feu")
print_calibration_table(results, 2, "Grand Feu")

# --- PARTIE C : ANALYSE DE RISQUE (SEUIL 50% GRAND FEU) ---
print("\n" + "="*90)
print(" ANALYSE DE RISQUE : PROBABILITÉ D'ÊTRE UN GRAND FEU > 50%")
print("="*90)

# 1. Pour les PETITS feux (True_Class == 0)
subset_petit = results[results['True_Class'] == 0]
nb_petit_alert = (subset_petit['Prob_Class_2'] > 0.5).sum()
pct_petit_alert = (nb_petit_alert / len(subset_petit)) * 100 if len(subset_petit) > 0 else 0
print(f"Petits Feux avec > 50% de risque Grand Feu : {pct_petit_alert:6.2f}%  ({nb_petit_alert}/{len(subset_petit)})")

# 2. Pour les MOYENS feux (True_Class == 1)
subset_moyen = results[results['True_Class'] == 1]
nb_moyen_alert = (subset_moyen['Prob_Class_2'] > 0.5).sum()
pct_moyen_alert = (nb_moyen_alert / len(subset_moyen)) * 100 if len(subset_moyen) > 0 else 0
print(f"Moyens Feux avec > 50% de risque Grand Feu : {pct_moyen_alert:6.2f}%  ({nb_moyen_alert}/{len(subset_moyen)})")

# 3. Pour les GRANDS feux (True_Class == 2) - C'est le Recall au seuil 0.5
subset_grand = results[results['True_Class'] == 2]
nb_grand_alert = (subset_grand['Prob_Class_2'] > 0.5).sum()
pct_grand_alert = (nb_grand_alert / len(subset_grand)) * 100 if len(subset_grand) > 0 else 0
print(f"Grands Feux avec > 50% de risque Grand Feu : {pct_grand_alert:6.2f}%  ({nb_grand_alert}/{len(subset_grand)})")
print("-" * 90)         

--- 1. PRÉPARATION DES DONNÉES ---
Features utilisées : 50
Features catégorielles détectées : 7
Train size: (58576, 50) | Test size: (14645, 50)

>>> STRATÉGIE DE POIDS APPLIQUÉE : [Petit: 1, Moyen: 5, Grand: 20]

--- Entraînement du modèle CatBoost ---
0:	test: 0.6363910	best: 0.6363910 (0)	total: 347ms	remaining: 5m 47s
100:	test: 0.8305164	best: 0.8305164 (99)	total: 18.1s	remaining: 2m 41s
200:	test: 0.8504625	best: 0.8504625 (200)	total: 39.5s	remaining: 2m 37s
300:	test: 0.8537572	best: 0.8537572 (300)	total: 1m 4s	remaining: 2m 29s
400:	test: 0.8554756	best: 0.8556330 (394)	total: 1m 28s	remaining: 2m 12s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8556329933
bestIteration = 394

Shrink model to first 395 iterations.

 RÉSULTATS DÉTAILLÉS MULTI-CLASSES (OPTIMISÉS SÉCURITÉ)
AUC Global (Weighted) : 0.8621
Accuracy Globale      : 0.8264
Recall Grand Feu      : 60.14% (Priorité absolue)

--- TOP 10 ALERTES (Triées par probabilité de Grand Feu) ---
P(Petit)   

On a avec ce modèle, le catboost des très bon résultat, une AUC moins bonne que sur les deux modèles précedents 0.86 mais une detection des grands feu qui est bien meilleur avec un recall des grands feu à 60%

In [6]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, recall_score, classification_report, confusion_matrix

# --- Import du nettoyage ---
try:
    from catclean import without_encode
except ImportError:
    # Fallback si le fichier n'est pas là pour le test
    def without_encode(df): return df
    print("ATTENTION : 'catclean.py' introuvable. Mode dégradé.")

# --- 1. CONFIGURATION DES FEATURES ---
BEST_FEATURES = [
    'Popo_1km', 'STATE', 'Population', 'EVH', 'EVC', 'rpms_1km', 'OWNER_DESCR', 
    'FOD_ID', 'Discovery_Time_Hours', 'NWCG_GENERAL_CAUSE', 'Region_Cluster', 
    'EBLR_PFS', 'EALR_PFS', 'No_FireStation_20.0km', 'Wind_x_Dryness', 'Land_Cover', 
    'EVT', 'EPL_AGE17', 'LATITUDE', 'GHM', 'NWCG_CAUSE_CLASSIFICATION', 'EPL_MOBILE', 
    'Ecoregion_US_L3CODE', 'NDVI-1day', 'Ecoregion_NA_L2CODE', 'Mang_Type', 'GDP', 
    'pr_Normal', 'EPL_NOHSDP', 'PM25F_PFS', 'bi_Percentile', 'rmin', 'sph', 
    'Annual_precipitation', 'HBF_PFS', 'vpd_Normal', 'Slope_x_Wind', 'sph_Normal', 
    'Slope_1km', 'rmax_Normal', 'bi_Normal', 'TRACT', 'Elevation', 'RPL_THEME4', 
    'EPLR_PFS', 'MHVF_PFS', 'RPL_THEME2', 'SDI', 'Wind_x_Potential', 'RMP_PFS'
]

# --- 2. PARAMÈTRES DU TRIAL 12 ---
# Poids extraits : Petit=1 (fixe), Moyen=5.48, Grand=70.37
TRIAL_12_WEIGHTS = [1, 5.48189108046529, 70.3743330902225]

TRIAL_12_PARAMS = {
    'iterations': 1000,          
    'learning_rate': 0.01211508050413514,   
    'depth': 4,                  
    'l2_leaf_reg': 1.93541594822678,        
    'bagging_temperature': 0.015055503993032683,
    'random_strength': 7.431601452326245,    
    'border_count': 209,         
    
    # Paramètres fixes
    'loss_function': 'MultiClass',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'allow_writing_files': False,
    'verbose': 100,
    'class_weights': TRIAL_12_WEIGHTS
}

def get_data_prepared(filename):
    print(f"--- CHARGEMENT ET NETTOYAGE ({filename}) ---")
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"ERREUR : '{filename}' introuvable.")
        exit()

    df_clean = without_encode(df)

    mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}
    if 'FIRE_SIZE_CLASS' in df_clean.columns:
        df_clean = df_clean.dropna(subset=['FIRE_SIZE_CLASS'])
        val_sample = df_clean['FIRE_SIZE_CLASS'].iloc[0]
        if isinstance(val_sample, str) and val_sample in mapping:
             df_clean['FIRE_SIZE_CLASS'] = df_clean['FIRE_SIZE_CLASS'].map(mapping)
        df_clean['FIRE_SIZE_CLASS'] = df_clean['FIRE_SIZE_CLASS'].astype(int)

    existing_features = [f for f in BEST_FEATURES if f in df_clean.columns]
    X = df_clean[existing_features]
    y = df_clean['FIRE_SIZE_CLASS']
    return X, y

def print_calibration_table(df_results, class_id, class_name):
    col_score = f'Prob_Class_{class_id}'
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    df_results['Tranche'] = pd.cut(df_results[col_score], bins=bins, labels=labels)

    print(f"\n>>> FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    counts = df_results['Tranche'].value_counts(sort=False)
    for label in labels:
        if label in counts.index and counts[label] > 0:
            subset = df_results[df_results['Tranche'] == label]
            real_percentage = (subset['True_Class'] == class_id).mean() * 100
            print(f"{label:<20} | {counts[label]:<10} | {real_percentage:.1f}%")
        else:
            print(f"{label:<20} | 0          | -")

# ==============================================================================
# MAIN PROGRAM
# ==============================================================================
if __name__ == "__main__":
    filename = "2020_FPA_FOD_cons.csv"
    X, y = get_data_prepared(filename)

    cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Split Stratifié
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n>>> CONFIGURATION TRIAL 12")
    print(f"Poids appliqués : {TRIAL_12_WEIGHTS}")
    print(f"Profondeur Arbre: {TRIAL_12_PARAMS['depth']}")
    print(f"Learning Rate   : {TRIAL_12_PARAMS['learning_rate']:.5f}")

    # Entraînement
    print("\n--- Entraînement du modèle (Trial 12) ---")
    model = CatBoostClassifier(**TRIAL_12_PARAMS, cat_features=cat_features)
    
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test),
        early_stopping_rounds=50
    )

    # Prédictions
    print("\n" + "="*90)
    print(" RÉSULTATS DÉTAILLÉS - CONFIGURATION TRIAL 12")
    print("="*90)
    
    probas = model.predict_proba(X_test)
    preds = model.predict(X_test)
    
    # Scores
    auc = roc_auc_score(y_test, probas, multi_class='ovr', average='weighted')
    recalls = recall_score(y_test, preds, average=None, zero_division=0)
    
    print(f"AUC Global (Weighted) : {auc:.4f}")
    print(f"Recall Petit Feu      : {recalls[0]:.2%}")
    print(f"Recall Moyen Feu      : {recalls[1]:.2%}")
    print(f"Recall Grand Feu      : {recalls[2]:.2%}  <-- SCORE CIBLE (OPTIMISÉ)")

    # Matrice de confusion simplifiée
    cm = confusion_matrix(y_test, preds)
    print("\nMatrice de Confusion (Lignes=Vrai, Colonnes=Prédictions) :")
    print(cm)

    # Analyse détaillée
    results = X_test.copy()
    results['True_Class'] = y_test
    results['Predicted_Class'] = preds
    results['Prob_Class_0'] = probas[:, 0]
    results['Prob_Class_1'] = probas[:, 1]
    results['Prob_Class_2'] = probas[:, 2]

    # --- TOP 10 ALERTES ---
    print("\n--- TOP 10 ALERTES (Triées par probabilité de Grand Feu) ---")
    top_alerts = results.sort_values(by='Prob_Class_2', ascending=False).head(10)
    print(f"{'P(Petit)':<10} | {'P(Moyen)':<10} | {'P(Grand)':<10} | {'VRAI':<6} | {'ÉTAT':<6} | {'CAUSE'}")
    print("-" * 90)
    for idx, row in top_alerts.iterrows():
        p0, p1, p2 = row['Prob_Class_0'], row['Prob_Class_1'], row['Prob_Class_2']
        cause = str(row['NWCG_GENERAL_CAUSE'])[:20]
        state = str(row['STATE'])
        true_c = str(row['True_Class'])
        print(f"{p0:.1%}      | {p1:.1%}      | {p2:.1%}      | {true_c:<6} | {state:<6} | {cause}")

    # --- FIABILITÉ ---
    print_calibration_table(results, 2, "Grand Feu")

    # --- ANALYSE DE RISQUE ---
    print("\n--- FAUSSES ALERTES ---")
    # Combien de fois a-t-on prédit Grand Feu alors que c'était un Petit Feu ?
    false_alarms = results[(results['Predicted_Class'] == 2) & (results['True_Class'] == 0)]
    print(f"Nombre de Petits Feux classés comme 'Grands' : {len(false_alarms)}")
    if len(results[results['True_Class'] == 0]) > 0:
        rate = len(false_alarms) / len(results[results['True_Class'] == 0])
        print(f"Taux de fausses alertes sur les Petits Feux : {rate:.2%}")

--- CHARGEMENT ET NETTOYAGE (2020_FPA_FOD_cons.csv) ---


C:\Users\axjui\AppData\Local\Temp\ipykernel_20032\1220696780.py:53: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Pipeline de Nettoyage (Mode CatBoost) - Démarrage. Dimensions : (73221, 308)
Colonnes avec plus de 50.0% de NaN à supprimer : 23
Colonnes constantes à supprimer : 8
Total de colonnes supprimées (Step 1) : 24
--- DÉBUT DU NETTOYAGE DÉTAILLÉ (handle_aberrant_values) ---
   -> 225 colonnes float vérifiées.
   -> 92 valeurs techniques (-1e30) remplacées par NaN.
   -> 3409 codes d'erreur (< -900) remplacés par NaN.
   -> 1474 erreurs topo (32767) remplacées par NaN.
Total des cellules corrigées : 4975
-> Features temporelles (Mois, Jour) extraites.
-> Conversion des colonnes 'faux-textes' en numériques...
-> Colonnes catégorielles supprimées (Manuelles) : 15
-> Imputation des NaN catégoriels par 'Missing' sur 22 colonnes.
Dimensions avant traitement des NaN : (73221, 271)
   -> 'DISCOVERY_TIME' transformée en 'Discovery_Time_Hours'.
-> Colonnes supprimées (Leakage/IDs) : 4
-> Sauvetage des colonnes critiques : ['EVH_1km', 'EVT_1km', 'EVC_1km', 'CH_1km', 'Land_Cover_1km']
   -> EVH_1km : 73

In [7]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, recall_score, confusion_matrix, classification_report

# --- Import du nettoyage ---
try:
    from catclean import without_encode
except ImportError:
    print("ATTENTION : 'catclean.py' introuvable. Mode dégradé.")
    def without_encode(df): return df

# --- 1. LES 50 VARIABLES ÉLUES ---
BEST_FEATURES = [
    'Popo_1km', 'STATE', 'Population', 'EVH', 'EVC', 'rpms_1km', 'OWNER_DESCR', 
    'FOD_ID', 'Discovery_Time_Hours', 'NWCG_GENERAL_CAUSE', 'Region_Cluster', 
    'EBLR_PFS', 'EALR_PFS', 'No_FireStation_20.0km', 'Wind_x_Dryness', 'Land_Cover', 
    'EVT', 'EPL_AGE17', 'LATITUDE', 'GHM', 'NWCG_CAUSE_CLASSIFICATION', 'EPL_MOBILE', 
    'Ecoregion_US_L3CODE', 'NDVI-1day', 'Ecoregion_NA_L2CODE', 'Mang_Type', 'GDP', 
    'pr_Normal', 'EPL_NOHSDP', 'PM25F_PFS', 'bi_Percentile', 'rmin', 'sph', 
    'Annual_precipitation', 'HBF_PFS', 'vpd_Normal', 'Slope_x_Wind', 'sph_Normal', 
    'Slope_1km', 'rmax_Normal', 'bi_Normal', 'TRACT', 'Elevation', 'RPL_THEME4', 
    'EPLR_PFS', 'MHVF_PFS', 'RPL_THEME2', 'SDI', 'Wind_x_Potential', 'RMP_PFS'
]

# --- 2. NOUVELLE CONFIGURATION (V2 - CONVERGENCE & ÉQUILIBRE) ---
# Poids modifiés : Petit=1, Moyen=12 (Boosté), Grand=70 (Sécurité max)
NEW_WEIGHTS = [1, 12, 70]

PARAMS_V2 = {
    'iterations': 3000,             # AUGMENTÉ (x3) pour laisser finir l'apprentissage
    'learning_rate': 0.012115,      # On garde ce LR lent et précis
    'depth': 4,                     # On garde la profondeur 4 (évite l'overfitting)
    'l2_leaf_reg': 1.935,
    'bagging_temperature': 0.015,
    'random_strength': 7.43,
    'border_count': 209,
    
    # Paramètres fixes
    'loss_function': 'MultiClass',
    'eval_metric': 'AUC',
    'random_seed': 42,
    'allow_writing_files': False,
    'verbose': 200,                 # Affiche les logs toutes les 200 itérations
    'class_weights': NEW_WEIGHTS,
    'od_type': 'Iter',              # Overfitting Detector
    'od_wait': 200                  # S'arrête si pas d'amélioration pendant 200 itérations
}

def get_data_prepared(filename):
    print(f"--- CHARGEMENT ({filename}) ---")
    try:
        df = pd.read_csv(filename)
    except FileNotFoundError:
        print(f"ERREUR : '{filename}' introuvable.")
        exit()

    df_clean = without_encode(df)

    mapping = {'A': 0, 'B': 0, 'C': 1, 'D': 2, 'E': 2, 'F': 2, 'G': 2}
    if 'FIRE_SIZE_CLASS' in df_clean.columns:
        df_clean = df_clean.dropna(subset=['FIRE_SIZE_CLASS'])
        val = df_clean['FIRE_SIZE_CLASS'].iloc[0]
        if isinstance(val, str) and val in mapping:
             df_clean['FIRE_SIZE_CLASS'] = df_clean['FIRE_SIZE_CLASS'].map(mapping)
        df_clean['FIRE_SIZE_CLASS'] = df_clean['FIRE_SIZE_CLASS'].astype(int)

    existing_features = [f for f in BEST_FEATURES if f in df_clean.columns]
    X = df_clean[existing_features]
    y = df_clean['FIRE_SIZE_CLASS']
    return X, y

def print_calibration_table(df_results, class_id, class_name):
    col_score = f'Prob_Class_{class_id}'
    bins = [0, 0.2, 0.4, 0.6, 0.8, 1.0]
    labels = ['0-20%', '20-40%', '40-60%', '60-80%', '80-100%']
    df_results['Tranche'] = pd.cut(df_results[col_score], bins=bins, labels=labels)

    print(f"\n>>> FIABILITÉ : {class_name.upper()} (Classe {class_id})")
    print(f"{'Tranche de Proba':<20} | {'Nb Feux':<10} | {'% Réellement ' + class_name}")
    print("-" * 65)

    counts = df_results['Tranche'].value_counts(sort=False)
    for label in labels:
        if label in counts.index and counts[label] > 0:
            subset = df_results[df_results['Tranche'] == label]
            real_percentage = (subset['True_Class'] == class_id).mean() * 100
            print(f"{label:<20} | {counts[label]:<10} | {real_percentage:.1f}%")
        else:
            print(f"{label:<20} | 0          | -")

# ==============================================================================
# MAIN PROGRAM
# ==============================================================================
if __name__ == "__main__":
    filename = "2020_FPA_FOD_cons.csv"
    X, y = get_data_prepared(filename)

    cat_features = X.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Split Stratifié
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )
    
    print(f"\n>>> CONFIGURATION V2 (LONG RUN)")
    print(f"Poids appliqués : {NEW_WEIGHTS} (Moyen Feu boosté)")
    print(f"Iterations      : {PARAMS_V2['iterations']} (Pour convergence)")

    # Entraînement
    print("\n--- Entraînement en cours (patience requise...) ---")
    model = CatBoostClassifier(**PARAMS_V2, cat_features=cat_features)
    
    model.fit(
        X_train, y_train,
        eval_set=(X_test, y_test)
        # early_stopping géré via od_wait dans les params
    )

    # Prédictions
    print("\n" + "="*90)
    print(" RÉSULTATS DÉTAILLÉS - MODELE V2")
    print("="*90)
    
    preds = model.predict(X_test)
    probas = model.predict_proba(X_test)
    
    # Scores
    auc = roc_auc_score(y_test, probas, multi_class='ovr', average='weighted')
    recalls = recall_score(y_test, preds, average=None, zero_division=0)
    
    print(f"AUC Global (Weighted) : {auc:.4f}")
    print(f"Recall Petit Feu      : {recalls[0]:.2%}")
    print(f"Recall Moyen Feu      : {recalls[1]:.2%}  (Doit augmenter)")
    print(f"Recall Grand Feu      : {recalls[2]:.2%}  (Doit rester haut)")

    # Matrice de confusion
    cm = confusion_matrix(y_test, preds)
    print("\nMatrice de Confusion (Lignes=Vrai, Colonnes=Prédictions) :")
    print(cm)
    
    # Analyse détaillée pour calibration
    results = X_test.copy()
    results['True_Class'] = y_test
    results['Predicted_Class'] = preds.flatten()
    results['Prob_Class_2'] = probas[:, 2]

    # --- FIABILITÉ ---
    print_calibration_table(results, 2, "Grand Feu")

    # --- ANALYSE DE RISQUE (Fausses Alertes) ---
    print("\n--- FAUSSES ALERTES (Pression sur les secours) ---")
    false_alarms = results[(results['Predicted_Class'] == 2) & (results['True_Class'] == 0)]
    nb_petits = len(results[results['True_Class'] == 0])
    rate = len(false_alarms) / nb_petits if nb_petits > 0 else 0
    
    print(f"Petits Feux classés 'Grands' : {len(false_alarms)} / {nb_petits}")
    print(f"Taux de fausses alertes      : {rate:.2%} (Cible : < 35%)")

--- CHARGEMENT (2020_FPA_FOD_cons.csv) ---


C:\Users\axjui\AppData\Local\Temp\ipykernel_20032\794479482.py:54: DtypeWarning: Columns (14,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filename)


Pipeline de Nettoyage (Mode CatBoost) - Démarrage. Dimensions : (73221, 308)
Colonnes avec plus de 50.0% de NaN à supprimer : 23
Colonnes constantes à supprimer : 8
Total de colonnes supprimées (Step 1) : 24
--- DÉBUT DU NETTOYAGE DÉTAILLÉ (handle_aberrant_values) ---
   -> 225 colonnes float vérifiées.
   -> 92 valeurs techniques (-1e30) remplacées par NaN.
   -> 3409 codes d'erreur (< -900) remplacés par NaN.
   -> 1474 erreurs topo (32767) remplacées par NaN.
Total des cellules corrigées : 4975
-> Features temporelles (Mois, Jour) extraites.
-> Conversion des colonnes 'faux-textes' en numériques...
-> Colonnes catégorielles supprimées (Manuelles) : 15
-> Imputation des NaN catégoriels par 'Missing' sur 22 colonnes.
Dimensions avant traitement des NaN : (73221, 271)
   -> 'DISCOVERY_TIME' transformée en 'Discovery_Time_Hours'.
-> Colonnes supprimées (Leakage/IDs) : 4
-> Sauvetage des colonnes critiques : ['EVH_1km', 'EVT_1km', 'EVC_1km', 'CH_1km', 'Land_Cover_1km']
   -> EVH_1km : 73